## Load Module Files from GitHub

In [ ]:
!rm /content/*.py*

rm: cannot remove '/content/*.py*': No such file or directory


In [ ]:
queries = []

filenames = [
             "GraphAttentionHead.py",
             "MultiHeadGraphAttention.py",
             "MultiHeadGraphAttentionLSTMCell.py",
             "mhga_utils.py"
]

for fn in filenames:
  queries.append(f"https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/{fn}")

queries.append("https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/imports.py")

In [ ]:
queries

['https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/GraphAttentionHead.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/MultiHeadGraphAttention.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/MultiHeadGraphAttentionLSTMCell.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/mhga_utils.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/imports.py']

In [ ]:
import subprocess
# Download and import my module files from GitHub
result = subprocess.run(["wget", "--no-cache", "--backups=1"] + queries, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
print(result.stderr.decode("utf-8"))

--2022-07-28 16:30:59--  https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/GraphAttentionHead.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2327 (2.3K) [text/plain]
Failed to rename GraphAttentionHead.py to GraphAttentionHead.py.1: (2) No such file or directory
Saving to: ‘GraphAttentionHead.py’

     0K ..                                                    100% 27.8M=0s

2022-07-28 16:30:59 (27.8 MB/s) - ‘GraphAttentionHead.py’ saved [2327/2327]

--2022-07-28 16:30:59--  https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/MultiHeadGraphAttention.py
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response.

## Load and Create Google Drive Linked Environment

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
folder_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/"
flight_data_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/flight_data/aggregated_flight_map_data.csv"
covid_data_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/US_time_series_covid_cases.csv"

## Load Sequential Data from Google Drive

In [ ]:
from mhga_utils import *

In [ ]:
NUM_DAYS_PER_SAMPLE = 7

In [ ]:
(formatted_X, formatted_adj_mat, formatted_y_infection, formatted_y_death) = load_sequential_data(covid_data_path, flight_data_path, num_days_per_sample = NUM_DAYS_PER_SAMPLE)

Generating Sequential Data...


100%|██████████| 793/793 [00:21<00:00, 36.90it/s]


## Scale and Split Train Test

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize

In [ ]:
# MinMax Scale by column
X_max = formatted_X.reshape(-1, formatted_X.shape[-1]).max(axis = 0)
X_min = formatted_X.reshape(-1, formatted_X.shape[-1]).min(axis = 0)
X_scaled = (formatted_X - X_min) / (X_max - X_min)

X_scaled.shape

(793, 7, 49, 5)

In [ ]:
# Standard Scale by column


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled.reshape(793, 7, -1), formatted_y_infection, train_size = .8)

## Create LSTM Model

In [ ]:
MODEL_ARCHITECTURE = (16, 16, 16)
SEQUENCE_LENGTH = 49
OUTPUT_SIZE = 49

In [ ]:
seq = tf.keras.models.Sequential()
seq.add(tf.keras.layers.Input(shape = (NUM_DAYS_PER_SAMPLE, SEQUENCE_LENGTH * 5)))
for i in range(len(MODEL_ARCHITECTURE) - 1):
  seq.add(tf.keras.layers.LSTM(MODEL_ARCHITECTURE[i], return_sequences = True))
  seq.add(tf.keras.layers.LeakyReLU())
  seq.add(tf.keras.layers.LayerNormalization())
seq.add(tf.keras.layers.LSTM(MODEL_ARCHITECTURE[-1], return_sequences = False))
seq.add(tf.keras.layers.LeakyReLU())
seq.add(tf.keras.layers.LayerNormalization())
seq.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation = "linear"))

In [ ]:
seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 16)             16768     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 7, 16)             0         
                                                                 
 layer_normalization (LayerN  (None, 7, 16)            32        
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 7, 16)             2112      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 7, 16)             0         
                                                                 
 layer_normalization_1 (Laye  (None, 7, 16)            32        
 rNormalization)                                        

## Train Model

In [ ]:
optimizer = tf.keras.optimizers.Adam()
BATCH_SIZE = 16
EPOCHS = 20

In [ ]:
@tf.function
def loss_func(y_true, y_pred):
  squared_diff = tf.square(y_true - y_pred)
  mse = tf.reduce_mean(squared_diff, axis=-1)
  squared_diff_mean = tf.square(tf.reduce_mean(y_true, axis = -1) - tf.reduce_mean(y_pred, axis = -1))
  return mse + squared_diff_mean

In [ ]:
seq.compile(
    loss = loss_func,
    optimizer = optimizer,
    metrics = ["mse", "mae"]
)

In [ ]:
seq.fit(
    x = X_train,
    y = y_train,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS)

Epoch 1/20
40/40 [==============================] - 10s 8ms/step - loss: 0.1780 - mse: 0.1717 - mae: 0.3039
Epoch 2/20
40/40 [==============================] - 0s 8ms/step - loss: 0.0195 - mse: 0.0158 - mae: 0.0892
Epoch 3/20
40/40 [==============================] - 0s 8ms/step - loss: 0.0071 - mse: 0.0044 - mae: 0.0488
Epoch 4/20
40/40 [==============================] - 0s 8ms/step - loss: 0.0050 - mse: 0.0036 - mae: 0.0435
Epoch 5/20
40/40 [==============================] - 0s 8ms/step - loss: 0.0033 - mse: 0.0027 - mae: 0.0366
Epoch 6/20
40/40 [==============================] - 0s 8ms/step - loss: 0.0022 - mse: 0.0019 - mae: 0.0305
Epoch 7/20
40/40 [==============================] - 0s 8ms/step - loss: 0.0014 - mse: 0.0013 - mae: 0.0254
Epoch 8/20
40/40 [==============================] - 0s 8ms/step - loss: 9.5348e-04 - mse: 8.8604e-04 - mae: 0.0208
Epoch 9/20
40/40 [==============================] - 0s 8ms/step - loss: 6.4487e-04 - mse: 6.0871e-04 - mae: 0.0174
Epoch 10/20
40/40 [=

In [ ]:
eval_res = seq.evaluate(
    x = X_test,
    y = y_test,
    batch_size = BATCH_SIZE)

10/10 [==============================] - 1s 4ms/step - loss: 7.7993e-05 - mse: 7.5155e-05 - mae: 0.0067


In [ ]:
eval_res

[7.79928159317933e-05, 7.515457400586456e-05, 0.006734113208949566]